# problem1

In [1]:
# prepress the data 
!python preprocess.py train.trees > train.trees.pre
#run train.trees.pre through unknown.py save the output to train.trees.pre.unk
!python unknown.py train.trees.pre > train.trees.pre.unk

In [2]:
#learn a probabilistic CFG from trees

In [3]:
!python tree.py < train.trees.pre.unk > grammer.txt

In [4]:
#solve the grammer remove /n and blank line
f = open("grammer.txt",'r')
g = open("new_grammer.txt",'w')
try:   
    while True:        
        line=f.readline()        
        if len(line)==0:           
            break        
        if line.count('\n')==len(line):            
            continue        
        g.write(line)
finally:    
    f.close()    
    g.close()

In [2]:
#use a dict key is every grammer and value is responding counter name 
#sort the dict and we could get the most popular grammer
grammer_dict = {}
file = open("new_grammer.txt")
for line in file.readlines():
    line=line.strip('\n')
    if(grammer_dict.has_key(line)):
        grammer_dict[line] = grammer_dict.get(line)+1
    else:
        grammer_dict[line] = 1
sorted(grammer_dict.items(),key = lambda x:x[1])
print "the grammar has been transfered into this form"
index = max(grammer_dict,key=grammer_dict.get)
for v,k in grammer_dict.items():
    print v,k


the grammar has been transfered into this form
NP_PRP -> me 82
VB -> leave 4
INTJ_UH -> Thanks 2
NNP -> T 4
NNP -> W 6
NNP -> P 1
NNP -> S 3
FRAG_WHNP -> WHNP* PP 32
NNP -> N 2
NN -> stop 7
NNP -> K 4
NNP -> J 4
NNP -> D 5
NNP -> F 6
NNP -> A 10
TOP -> FRAG_NP_NNP PUNC 1
NNP -> C 6
S_VP -> VBP PP 2
NP -> NP* NNS 34
NP -> NP* NNP 30
NP* -> NN NNP 6
WHNP -> WRB RB 5
NNP -> City 29
VBG -> Leaving 4
MD -> 'd 10
SYM -> <unk> 3
NNP -> U 4
S_VP -> VP* INTJ_UH 2
WP -> What 3
SQ* -> NP PP 1
NP -> NN NN 9
S_VP -> VB NP_PRP 2
VP -> VBG NP 6
NP -> NP* CD 17
NP -> JJS NN 8
S_VP -> VP* NP 73
NN -> airfare 5
PP_IN -> from 1
WHNP_WP -> What 20
POS -> 's 2
NN -> flight 75
VBZ -> Does 7
VBN -> <unk> 1
SQ -> VBZ NP 22
PP -> TO NP 58
FRAG_NP -> NP_NNS PP 7
INTJ_UH -> oh 1
VP* -> VP CC 7
NP* -> NP NN 1
FRAG_NP_NN -> <unk> 2
NP* -> NP NP 2
DT -> Any 1
NNP -> Petersburg 3
VB -> fly 5
SQ_VP -> VBZ NP 1
NP -> JJ NN 11
ADVP -> RB PP 1
VBZ -> is 31
FRAG* -> WHNP PP 2
VB -> serve 7
NP_NNP -> Denver 15
DT -> the 1

There are 752 rules in my grammer ,the most frequent rule is PUNC -> . it occurs 346 times

# problem 2

In [6]:
import copy
# transfer the rules with frequency into the probability
count_sum = 0
for key in grammer_dict.keys():
    count_sum = count_sum + grammer_dict.get(key)
#count_sum is the total number of the rules
print count_sum

7561


In [7]:
from math import log
#grammer_prob is a grammer with probablity
grammer_prob = copy.deepcopy(grammer_dict)
for key in grammer_prob.keys():
    grammer_prob[key] = log(float(grammer_prob[key])/float(count_sum),10)
#grammer_prob is a dict whose key are rules and values are corresponding probability\

#word_search_children is a dict whose key is string after -> value is rule
#word_search_parent is a dict whose key is rule -> value is string before ->
#grammer_prob is a dict whose key is rule -> value is probability
from collections import defaultdict
word_search_children = defaultdict(list)
for k, v in grammer_prob.items():
    word_search_children[k.split("->")[-1].strip()].append(k)
    
word_search_parent = defaultdict(list) 
for k,v in grammer_prob.items():
    word_search_parent[k].append(k.split("->")[0].strip())

In [8]:
#transfer the sentence into seperate word

def to_word_list(sentence):
    word_list = sentence.split(" ")
    return word_list

In [12]:
#write a parsing and take the grammer and senctence as input, and output the highest_probability parse
import copy
def initial_table(sentence,parent_list,children_list,rule_prob):
    l = to_word_list(sentence)
    length = len(l)
    table = [[[] for i in range(length)] for i in range(length)] 
    table_prob = copy.deepcopy(table)
    for i in range(len(l)):
        grammer_key = children_list[l[i]]
        for word in grammer_key:
            table_prob[i][i].append(rule_prob[word])
            table[i][i]= table[i][i]+parent_list[word]
    table_dict = {}
    table_dict["table"] = table
    table_dict["table_prob"] = table_prob
    return table_dict
result = initial_table(input_string,word_search_parent,word_search_children,grammer_prob) 
print result["table"]
print result["table_prob"]

#use initial_table we could get a table on the diagnol with the word of length1 and its label       

[[['DT'], [], [], [], [], [], [], []], [[], ['NN', 'NP_NN'], [], [], [], [], [], []], [[], [], ['MD'], [], [], [], [], []], [[], [], [], ['VB'], [], [], [], []], [[], [], [], [], ['CD'], [], [], []], [[], [], [], [], [], ['RB'], [], []], [[], [], [], [], [], [], ['NP_NN', 'NN'], []], [[], [], [], [], [], [], [], ['PUNC']]]
[[[-3.2765192467342565], [], [], [], [], [], [], []], [[], [-2.003517974670519, -3.4014579833425564], [], [], [], [], [], []], [[], [], [-3.2765192467342565], [], [], [], [], []], [[], [], [], [-3.5775492423982374], [], [], [], []], [[], [], [], [], [-3.5775492423982374], [], [], []], [[], [], [], [], [], [-2.9754892510702753], [], []], [[], [], [], [], [], [], [-3.100427987678575, -3.4014579833425564], []], [[], [], [], [], [], [], [], [-1.3395031392694425]]]


In [13]:
#use a funciton index_table it add a column to the original table in order to change index of it
def index_table(table):
    
    for l in table:
        l.insert(0,[])
    return table

print index_table(result["table"])
print index_table(result["table_prob"])
print len(result["table"])
print len(result['table'][0])

[[[], ['DT'], [], [], [], [], [], [], []], [[], [], ['NN', 'NP_NN'], [], [], [], [], [], []], [[], [], [], ['MD'], [], [], [], [], []], [[], [], [], [], ['VB'], [], [], [], []], [[], [], [], [], [], ['CD'], [], [], []], [[], [], [], [], [], [], ['RB'], [], []], [[], [], [], [], [], [], [], ['NP_NN', 'NN'], []], [[], [], [], [], [], [], [], [], ['PUNC']]]
[[[], [-3.2765192467342565], [], [], [], [], [], [], []], [[], [], [-2.003517974670519, -3.4014579833425564], [], [], [], [], [], []], [[], [], [], [-3.2765192467342565], [], [], [], [], []], [[], [], [], [], [-3.5775492423982374], [], [], [], []], [[], [], [], [], [], [-3.5775492423982374], [], [], []], [[], [], [], [], [], [], [-2.9754892510702753], [], []], [[], [], [], [], [], [], [], [-3.100427987678575, -3.4014579833425564], []], [[], [], [], [], [], [], [], [], [-1.3395031392694425]]]
8
9


In [17]:
#after all the initialization process we could implement the parse of a sentence
from math import log

def parsing_table(sentence,parent_list,children_list,rule_prob):
    table_with_prob = initial_table(sentence,parent_list,children_list,rule_prob)
    cky_table = index_table(table_with_prob["table"])
    cky_prob = index_table(table_with_prob["table_prob"])
    n = len(cky_table[0])
    for width in range(2,n):
        for start in range(0,n-width):
            end = start+width
            for mid in range(start+1,end):
                list_y = cky_table[start][mid]
                list_z = cky_table[mid][end]
                for y in list_y:
                    for z in list_z:
                        temp = [y,z]
                        child = " ".join(temp)
                        if(child in children_list):
                            grammer_list=children_list[child]
                            for grammer in grammer_list:
                                cky_table[start][end]=cky_table[start][end]+parent_list[grammer]
                                
    return cky_table
                                
table1 = parsing_table(input_string,word_search_parent,word_search_children,grammer_prob)
print table1[0][8]

['TOP', 'TOP', 'TOP', 'TOP', 'TOP', 'TOP', 'TOP', 'TOP', 'TOP', 'TOP', 'TOP', 'TOP', 'TOP', 'TOP', 'TOP', 'TOP']


In [15]:
#f_string is the first line of dev.strings, we transfer it into input_string and delete \n at the end
#of sentence and pass it into the to_word_list transfer it into a word list
f_string = open('dev.strings')
input_string = f_string.readlines()[0].strip('\n')
l =  to_word_list(input_string)
print l 
a = "the"
b = "flight"
s = [a,b]
t = " ".join(s)
print t

['The', 'flight', 'should', 'be', 'eleven', 'a.m', 'tomorrow', '.']
the flight


In [16]:
#word_Search is a hashmap whose key is the value after -> value is the whole corresponding rule


for i in range(len(l)):
    grammer_key = word_search_children[l[i]]
    print grammer_key
    for i in grammer_key:
        prob = grammer_prob[i]
        tag = [word_search_parent[i] for i in grammer_key]
        print prob
        print tag
    
    
                                                  
                                                  
        
    

['DT -> The']
-3.27651924673
[['DT']]
['NN -> flight', 'NP_NN -> flight']
-2.00351797467
[['NN'], ['NP_NN']]
-3.40145798334
[['NN'], ['NP_NN']]
['MD -> should']
-3.27651924673
[['MD']]
['VB -> be']
-3.5775492424
[['VB']]
['CD -> eleven']
-3.5775492424
[['CD']]
['RB -> a.m']
-2.97548925107
[['RB']]
['NP_NN -> tomorrow', 'NN -> tomorrow']
-3.10042798768
[['NP_NN'], ['NN']]
-3.40145798334
[['NP_NN'], ['NN']]
['PUNC -> .']
-1.33950313927
[['PUNC']]


In [ ]:
# problem 3